In [1]:
import csv
import sqlite3
import functions
import pandas as pd
import requests as rq
from config import api_key
# pip install geopy
from geopy.geocoders import Nominatim
import pickle

In [2]:
# df = pd.read_csv('Resources/airlines_reviews.csv')
# df.head(2)

In [3]:
# conn = sqlite3.connect('airline_passenger_satisfaction.sqlite')
# df.to_sql('airlines_reviews', conn, if_exists='replace')
# db = conn.cursor()
# db.execute('SELECT * FROM airlines_reviews').fetchall()

In [14]:
# Loading model to compare the results ### model = pickle.load(open('model.pkl','rb'))
df = pickle.load(open('df.pkl','rb'))
df

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Title Polarity,Title Subjectivity,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,0.600,0.9,True,False,False,False,False,False,False,True
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,-1.000,1.0,False,True,False,False,False,False,False,True
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,0.000,0.0,False,True,False,False,False,False,True,False
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.300,0.3,False,True,False,False,False,False,False,True
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.700,0.6,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,Korean Air,6,2016,5,4,5,3,4,7,1,...,0.000,0.0,False,True,False,False,True,False,False,False
8096,Korean Air,6,2016,3,5,5,4,5,10,1,...,0.000,0.0,False,True,False,False,False,True,False,False
8097,Korean Air,4,2016,4,5,5,5,1,2,0,...,0.000,0.0,True,False,False,False,True,False,False,False
8098,Korean Air,4,2016,5,1,3,4,5,10,1,...,0.200,0.1,True,False,False,False,True,False,False,False


In [15]:
print(f'Rows and Columns: {df.shape}')
df.columns

Rows and Columns: (7683, 31)


Index(['Airline', 'Month Flown', 'Year Flown', 'Seat Comfort', 'Staff Service',
       'Food & Beverages', 'Inflight Entertainment', 'Value For Money',
       'Overall Rating', 'Recommended', 'Origin', 'Destination', 'Via',
       'Origin_Code', 'Destination_Code', 'Via_Code', 'Origin_City',
       'Destination_City', 'Via_City', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure'],
      dtype='object')

In [17]:
unique_codes = df['Origin_Code'].unique()
len(unique_codes)
unique_codes

array(['CGK', 'HKT', 'SAI', 'SIN', 'BWN', 'LHR', 'PER', 'SYD', 'MIL',
       'LON', 'BKK', 'KUL', 'PEN', 'FRA', 'BNE', 'MEL', 'NYC', 'AKL',
       'JNB', 'MNL', 'BOM', 'ATH', 'CAN', 'KCZ', 'DXB', 'SFO', 'ADL',
       'CPT', 'EWR', 'HKG', 'YVR', 'PNH', 'DVO', 'DPS', 'HAN', 'MAN',
       'JFK', 'LAX', 'NRT', 'IST', 'SUB', 'PAR', 'BLR', 'AMS', 'SGN',
       'DAD', 'ROM', 'CCU', 'MUC', 'CMB', 'SAO', 'KIX', 'HND', 'ZRH',
       'UPG', 'AMD', 'CPH', 'PUS', 'SPK', 'PVG', 'DEL', 'DAC', 'SEA',
       'ARN', 'CHC', 'ICN', 'DUS', 'WLG', 'BCN', 'TPE', 'RGN', 'CJB',
       'MAA', 'IAH', 'CBR', 'DME', 'MXP', 'CDG', 'FCO', 'DOH', 'ORD',
       'NCE', 'TBS', 'BEY', 'LGW', 'DFW', 'KWI', 'LUN', 'BRU', 'RUH',
       'OSL', 'KTM', 'VIE', 'BWI', 'AUH', 'EBB', 'PHL', 'WAW', 'EDI',
       'ISB', 'GVA', 'YMQ', 'DAL', 'MPM', 'BUH', 'DUB', 'HRE', 'BOS',
       'MAD', 'IKA', 'SOF', 'LOS', 'PRG', 'NBO', 'ATL', 'ALG', 'SHJ',
       'MIA', 'ZAG', 'COK', 'ESB', 'KGL', 'BSR', 'MLE', 'YTO', 'TUN',
       'CAI', 'BJV',

In [18]:
unique_codes_Dest = df['Destination_Code'].unique()
len(unique_codes_Dest)
unique_codes

array(['CGK', 'HKT', 'SAI', 'SIN', 'BWN', 'LHR', 'PER', 'SYD', 'MIL',
       'LON', 'BKK', 'KUL', 'PEN', 'FRA', 'BNE', 'MEL', 'NYC', 'AKL',
       'JNB', 'MNL', 'BOM', 'ATH', 'CAN', 'KCZ', 'DXB', 'SFO', 'ADL',
       'CPT', 'EWR', 'HKG', 'YVR', 'PNH', 'DVO', 'DPS', 'HAN', 'MAN',
       'JFK', 'LAX', 'NRT', 'IST', 'SUB', 'PAR', 'BLR', 'AMS', 'SGN',
       'DAD', 'ROM', 'CCU', 'MUC', 'CMB', 'SAO', 'KIX', 'HND', 'ZRH',
       'UPG', 'AMD', 'CPH', 'PUS', 'SPK', 'PVG', 'DEL', 'DAC', 'SEA',
       'ARN', 'CHC', 'ICN', 'DUS', 'WLG', 'BCN', 'TPE', 'RGN', 'CJB',
       'MAA', 'IAH', 'CBR', 'DME', 'MXP', 'CDG', 'FCO', 'DOH', 'ORD',
       'NCE', 'TBS', 'BEY', 'LGW', 'DFW', 'KWI', 'LUN', 'BRU', 'RUH',
       'OSL', 'KTM', 'VIE', 'BWI', 'AUH', 'EBB', 'PHL', 'WAW', 'EDI',
       'ISB', 'GVA', 'YMQ', 'DAL', 'MPM', 'BUH', 'DUB', 'HRE', 'BOS',
       'MAD', 'IKA', 'SOF', 'LOS', 'PRG', 'NBO', 'ATL', 'ALG', 'SHJ',
       'MIA', 'ZAG', 'COK', 'ESB', 'KGL', 'BSR', 'MLE', 'YTO', 'TUN',
       'CAI', 'BJV',

In [19]:
unique_codes = set(unique_codes)
unique_codes_Dest = set(unique_codes_Dest)
unique_codes = unique_codes.union(unique_codes_Dest)
unique_codes

{'ABJ',
 'ABV',
 'ABZ',
 'ACC',
 'ADB',
 'ADD',
 'ADL',
 'AER',
 'AEX',
 'AGP',
 'AJA',
 'AKL',
 'ALA',
 'ALG',
 'AMD',
 'AMM',
 'AMS',
 'AOJ',
 'AQJ',
 'ARN',
 'ASR',
 'ATH',
 'ATL',
 'ATQ',
 'AUH',
 'AUS',
 'AXT',
 'AYT',
 'BAH',
 'BAK',
 'BCN',
 'BEG',
 'BER',
 'BEY',
 'BGO',
 'BGW',
 'BHM',
 'BHX',
 'BIA',
 'BIO',
 'BIQ',
 'BJL',
 'BJV',
 'BKK',
 'BKO',
 'BLL',
 'BLQ',
 'BLR',
 'BNE',
 'BOD',
 'BOG',
 'BOM',
 'BOS',
 'BRE',
 'BRU',
 'BSL',
 'BSR',
 'BUD',
 'BUE',
 'BUH',
 'BUS',
 'BWI',
 'BWN',
 'BXH',
 'CAI',
 'CAN',
 'CAS',
 'CAY',
 'CBR',
 'CBU',
 'CCJ',
 'CCS',
 'CCU',
 'CDG',
 'CEB',
 'CGK',
 'CGN',
 'CHC',
 'CJB',
 'CJU',
 'CKG',
 'CKY',
 'CLJ',
 'CMB',
 'CMN',
 'CND',
 'CNS',
 'CNX',
 'COK',
 'CPH',
 'CPT',
 'CRK',
 'CTA',
 'CTS',
 'CTU',
 'CUN',
 'CVG',
 'CWL',
 'DAC',
 'DAD',
 'DAL',
 'DAR',
 'DBV',
 'DEL',
 'DEN',
 'DET',
 'DFW',
 'DKR',
 'DLA',
 'DLC',
 'DLM',
 'DME',
 'DMM',
 'DOH',
 'DPS',
 'DTW',
 'DUB',
 'DUR',
 'DUS',
 'DVO',
 'DWC',
 'DXB',
 'DYU',
 'EAP',
 'EBB',


In [20]:
import json

# Load the JSON file containing airport information
with open('Resources/airport_data.json', 'r') as file:
    airport_data = json.load(file)

# Extract information for the 600 unique airport codes
airport_info = {}
for code in unique_codes:
    if code in airport_data:
        airport_info[code] = {
            'name': airport_data[code]['name'],
            'latitude': airport_data[code]['latitude'],
            'longitude': airport_data[code]['longitude']
        }

airport_info

{}

In [24]:
geolocator.geocode('MEX airport').latitude

19.434325299999998

In [34]:
Airports_Lat_Lng_df = pd.DataFrame()
for loc in unique_codes:
    Airports_Lat_Lng_df['airport_code'] = loc
    Airports_Lat_Lng_df['Lat'] = geolocator.geocode(f'{loc} airport').latitude
    Airports_Lat_Lng_df['Lng'] = geolocator.geocode(f'{loc} airport').longitude

Airports_Lat_Lng_df

AttributeError: 'NoneType' object has no attribute 'latitude'

In [32]:
len(set(df.Origin_Code))

365

In [10]:
custom_user_agent = "MyCustomUserAgent"
geolocator = Nominatim(user_agent="custom_user_agent")

In [ ]:
import time

In [27]:
df.columns

Index(['Airline', 'Month Flown', 'Year Flown', 'Seat Comfort', 'Staff Service',
       'Food & Beverages', 'Inflight Entertainment', 'Value For Money',
       'Overall Rating', 'Recommended', 'Origin', 'Destination', 'Via',
       'Origin_Code', 'Destination_Code', 'Via_Code', 'Origin_City',
       'Destination_City', 'Via_City', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure'],
      dtype='object')

In [ ]:
### using f" 'airport' added to airport code

def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(f"{airport_code} airport")
    print(airport_code, location)
    time.sleep(1)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

airport_pos = {code:get_lat_long(code) for code in unique_codes}
airport_pos

In [ ]:
### try output for unique_codes + lat + long

In [ ]:
# no f' - added for_loop for airport_pos_2 df

def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(airport_code)
    print(airport_code, location)
    time.sleep(1)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

airport_pos_2 = {}
for code in unique_codes:
    lat, long = get_lat_long(code)
    if lat is not None and long is not None:
        airport_pos_2[code] = (lat, long)

airport_pos_2

In [ ]:
#### failed attempt to get just lat_long from variable 'unique_codes'

def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(airport_code)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the function to create new columns for Latitude and Longitude
df['Latitude'], df['Longitude'] = zip(*df['Airport_Code'].apply(get_lat_long))

# Write the output to a CSV file
df.to_csv('airport_lat_long.csv', index=False)

# Display the updated DataFrame
print(df)

In [45]:
##### SUCCESSFUL for getting lat_long for subset BUT NOT for 8,000 rows


def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(airport_code)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the function to create new columns for Origin_Latitude, Origin_Longitude, Destination_Latitude, Destination_Longitude for the first 10 rows
df_subset = df.head(10)
df_subset['Origin_Latitude'], df_subset['Origin_Longitude'] = zip(*df_subset['Origin_Code'].apply(get_lat_long))
df_subset['Destination_Latitude'], df_subset['Destination_Longitude'] = zip(*df_subset['Destination_Code'].apply(get_lat_long))

# Display the updated DataFrame for the first 10 rows
print(df_subset)

              Airline           Class          Destination  Food & Beverages  \
0  Singapore Airlines  Business Class            Singapore                 4   
1  Singapore Airlines   Economy Class            Singapore                 4   
2  Singapore Airlines   Economy Class            Singapore                 2   
3  Singapore Airlines   Economy Class      London Heathrow                 5   
4  Singapore Airlines   Economy Class           Phnom Penh                 5   
5  Singapore Airlines   Economy Class               Cairns                 5   
6  Singapore Airlines   Economy Class            Frankfurt                 1   
7  Singapore Airlines   Economy Class  Bandar Seri Begawan                 3   
8  Singapore Airlines   Economy Class              Jakarta                 5   
9  Singapore Airlines   Economy Class            Singapore                 4   

   Inflight Entertainment  Month Flown     Origin  Overall Rating  Polarity  \
0                       4           12  

In [47]:
df.head()

,Airline,Class,Destination,Food & Beverages,Inflight Entertainment,Month Flown,Origin,Overall Rating,Polarity,Recommended,...,Via_2,Year Flown,Origin_Code,Destination_Code,Via_Code,Via_2_Code,Origin_City,Destination_City,Via_City,Via_2_City
0,Singapore Airlines,Business Class,Singapore,4,4,12,Jakarta,9,0.384667,yes,...,NaN,2023,CGK,SIN,None,NaN,Jakarta,Singapore,None,NaN
1,Singapore Airlines,Economy Class,Singapore,4,4,2,Phuket,3,-0.212500,no,...,NaN,2024,HKT,SIN,None,NaN,Phuket,Singapore,None,NaN
2,Singapore Airlines,Economy Class,Singapore,2,1,2,Siem Reap,10,0.500000,yes,...,NaN,2024,SAI,SIN,None,NaN,Siem Reap,Singapore,None,NaN
3,Singapore Airlines,Economy Class,London Heathrow,5,5,2,Singapore,10,0.325496,yes,...,NaN,2024,SIN,LHR,None,NaN,Singapore,London Heathrow,None,NaN
4,Singapore Airlines,Economy Class,Phnom Penh,5,5,2,Singapore,10,0.412500,yes,...,NaN,2024,SIN,PNH,None,NaN,Singapore,Phnom Penh,None,NaN


In [49]:
len(unique_Origin)

654

In [ ]:
print(df['Origin_Code'].dtype)
print(df['Destination_Code'].dtype)
print(df['Via_Code'].dtype)
print(df['Via_2_Code'].dtype)

In [ ]:
df['Origin_Code'] = df['Origin_Code'].astype('category')
df['Destination_Code'] = df['Destination_Code'].astype('category')
df['Via_Code'] = df['Via_Code'].astype('category')
df['Via_2_Code'] = df['Via_2_Code'].astype('category')

In [ ]:
print(df['Origin_Code'].dtype)
print(df['Destination_Code'].dtype)
print(df['Via_Code'].dtype)
print(df['Via_2_Code'].dtype)

consider optimizing your code further by looking into other ways to improve the efficiency of your operations, such as using vectorized operations, avoiding unnecessary iterations, and utilizing appropriate indexing techniques

Avoid unnecessary iterations
1. **Use Vectorized Operations**: Instead of iterating over rows, try to perform operations on entire columns or subsets of data using vectorized operations provided by libraries like NumPy and Pandas. These operations are optimized for performance and can significantly speed up your code.

2. **Avoid Iterating Over DataFrame Rows**: If possible, avoid iterating over rows using methods like `iterrows()` or `apply()`. These methods can be slow, especially on large datasets. Instead, try to find ways to apply operations to entire columns or use built-in Pandas functions that operate on the entire DataFrame.

3. **Use GroupBy**: If you need to perform operations on groups of data, consider using the `groupby()` function in Pandas. This allows you to group your data based on certain criteria and apply functions to each group without explicit iteration.

4. **Merge and Join Operations**: Instead of iterating over two DataFrames to perform operations, consider using merge and join operations provided by Pandas. These operations are optimized and can be more efficient than iterating over rows.

5. **List Comprehensions and Built-in Functions**: When possible, use list comprehensions or built-in functions to perform operations on lists or arrays. These are typically faster than explicit loops and can make your code more concise and readable.

6. **Utilize NumPy Functions**: NumPy provides a wide range of vectorized functions that can operate on arrays efficiently. Wherever possible, try to use NumPy functions instead of writing custom Python loops.

By following these tips and leveraging the capabilities of libraries like Pandas and NumPy, you can avoid unnecessary iterations and improve the performance of your code

In [ ]:
# Convert the column back to its original data type
df['Origin_Code'] = df['Origin_Code'].astype('original_data_type')
df['Destination_Code'] = df['Destination_Code'].astype('original_data_type')
df['Via_Code'] = df['Via_Code'].astype('original_data_type')
df['Via_2_Code'] = df['Via_2_Code'].astype('original_data_type')
print(df['Origin_Code'].dtype)

https://www.developer.aero/api-catalog/flight-duration-overview

In [17]:
####### DO NOT RUN !!!!!!!!!!
# unique_via = df['Via'].unique()
# airports_via = airport_codes['City'].tolist() + airport_codes['Code'].tolist()
# via_values_not_found = [via for via in unique_via if via not in airports_via]

# print("Values from the 'Via' column not found in airports_utf.csv 'City' or 'Code' columns:")
# print(via_values_not_found)

# Values from the 'Via' column not found in airports_utf.csv 'City' or 'Code' columns:
# [None, 'DOA', 'Lagoa', 'DXB/MLE', 'Минск', 'Panama city', '', 'paris', 'Italy', 'Lyons', 'Hongkong', 'Hong kong', 'Degeau', 'INC']

Values from the 'Via' column not found in airports_utf.csv 'City' or 'Code' columns:
[None, 'DOA', 'Lagoa', 'DXB/MLE', 'Минск', 'Panama city', '', 'paris', 'Italy', 'Lyons', 'Hongkong', 'Hong kong', 'Degeau', 'INC']


In [ ]:
### lookup failures, only return cities not found, if return series call unique.apply - return nothing if you can't find it

### get lats/long for origin-destination-via

### https://www.developer.aero/api-catalog/flight-duration-overview

In [ ]:
# keep routes & split into Origin & Destination, find coord for the locations
# Target "Recommended"
# maybe user can provide the following and 
# Type of Traveller', 'Route', 'Class', 'Seat Comfort',
# 'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
# 'Value For Money